In [1]:
%pylab inline
%matplotlib inline

import os
import time
import collections
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")

import paths
import my_plot
import emolex
import tokenizer


import useful_methods

Populating the interactive namespace from numpy and matplotlib


### One Game

In [ ]:
# choosing data
week = str(13)
GW = 'GW' + week
filenames = useful_methods.FolderFiles(GW + '/SingleGames', paths.READ_PATH_EXTRACTED_CSV, ends='.csv')
filenames

In [ ]:
# Set Game
filename = filenames[1]

# Read DF
os.chdir(paths.READ_PATH_EXTRACTED_CSV + GW + '/SingleGames')
df = useful_methods.csv_dic_df(filename)

# [Filter] DF
dfFilter = useful_methods.FilterDF(df[df.status != 'retweet'])

In [ ]:
# Emolex dic
%time dic_emolex_soccer, dic_emolex_stemmed_soccer = emolex.EmolexSoccerDic()

In [ ]:
# Add Columns: ['hash_emolex', 'emoji', 'emoticon']
%time dfFilter['hash_emolex'] = [tokenizer.HashEmolex(tags, dic_emolex_soccer) for tags in dfFilter.tags]
%time dfFilter['emoji'] = [tokenizer.EmojiText(text) for text in dfFilter.text]
%time dfFilter['emoticon'] = [tokenizer.EmoticonText(text) for text in dfFilter.text]

In [ ]:
print(
    "\nHash Emolex: ", len(dfFilter[dfFilter.hash_emolex != '']),
    "\nEmoji: ", len(dfFilter[dfFilter.emoji != '']),
    "\nEmoticon: ", len(dfFilter[dfFilter.emoticon != '']),
    "\n\nAll: ", len(dfFilter[(dfFilter.hash_emolex != '') | (dfFilter.emoji != '') | (dfFilter.emoticon != '')])
)

In [ ]:
dfHashEmolex = dfFilter[(dfFilter.hash_emolex != '')]

dfNewHashEmolex = pd.DataFrame(columns=['text', 'hash_emolex_word', 'sentiments'])
texts = []
hash_emolexs = []
sentiments = []

for i in range(len(dfHashEmolex)):
    # hash tags
    hash_emolex = dfHashEmolex.iloc[i]['hash_emolex'].split(',')
    
    # each tag
    for emo in set(hash_emolex):
        if emo in dic_emolex_soccer:
            texts.append(dfHashEmolex.iloc[i]['text'])
            hash_emolexs.append(emo)
            sentiment = ",".join([key for key, value in dic_emolex_soccer[emo].items() if value > 0])
            sentiments.append(sentiment)

dfNewHashEmolex['text'] = texts
dfNewHashEmolex['hash_emolex_word'] = hash_emolexs
dfNewHashEmolex['sentiments'] = sentiments

dfNewHashEmolex

### All Games

In [2]:
# Emolex dic
%time dic_emolex_soccer, dic_emolex_stemmed_soccer = emolex.EmolexSoccerDic()

All Words: 14152
CPU times: user 7.74 s, sys: 102 ms, total: 7.84 s
Wall time: 7.96 s


In [3]:
# All Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

# HT: Equal games
# dfGameInfosEqual = dfGameInfos[(dfGameInfos['score_ht_home'] == dfGameInfos['score_ht_away']) &
#                                (dfGameInfos['GW'] >= 4)].copy()

weeks = list(dfGameInfos['GW'])
home_teams = list(dfGameInfos['home_team'])
away_teams = list(dfGameInfos['away_team'])
score_ht_homes = list(dfGameInfos['score_ht_home'])
score_ht_aways = list(dfGameInfos['score_ht_away'])
score_ft_homes = list(dfGameInfos['score_ft_home'])
score_ft_aways = list(dfGameInfos['score_ft_away'])

In [4]:
# Define: All Hash Emolex DF
dfNewHashEmolex = pd.DataFrame(columns=['text', 'hash_emolex_word', 'sentiments'])
texts = []
hash_emolexs = []
sentiments = []

In [5]:
start_time = time.time()

# Start All Here
for index in range(len(weeks)):
    try:
        # Read Single Game as DF
        filename = home_teams[index] + '_vs_' + away_teams[index] + '.csv'
        os.chdir(paths.READ_PATH_EXTRACTED_CSV + 'GW' + str(weeks[index]) + '/SingleGames/')
        df = useful_methods.csv_dic_df(filename)

        
        ###########################################################################
        
        # Filter DF: remove tweets of stream, bots ...
        dfFilter = useful_methods.FilterDF(df[df.status != 'retweet'])
        
        # Add column: hash_emolex
        dfFilter['hash_emolex'] = [tokenizer.HashEmolex(tags, dic_emolex_soccer) for tags in dfFilter.tags]
        
        # Emolex in Hashtags
        dfHashEmolex = dfFilter[(dfFilter.hash_emolex != '')]

        # Check through each Tweet
        for i in range(len(dfHashEmolex)):
            # hash tags
            hash_emolex = dfHashEmolex.iloc[i]['hash_emolex'].split(',')

            # each tag
            for emo in set(hash_emolex):
                if emo in dic_emolex_soccer:
                    # appedings
                    texts.append(dfHashEmolex.iloc[i]['text'])
                    hash_emolexs.append(emo)
                    sentiment = ",".join([key for key, value in dic_emolex_soccer[emo].items() if value > 0])
                    sentiments.append(sentiment)
    except:
        continue

print("[Done]: %.2f" % (time.time() - start_time))

[Done]: 52.23


In [7]:
# Result
dfNewHashEmolex['text'] = texts
dfNewHashEmolex['hash_emolex_word'] = hash_emolexs
dfNewHashEmolex['sentiments'] = sentiments

dfNewHashEmolex

,text,hash_emolex_word,sentiments
0,#body #status Dewibet Jkt - Susunan Pemain… h...,status,positive
1,BRILLIANT!!!!! @Mo_Farah Simply the best!!!!! ...,athlete,positive
2,@markvanhartenn yesss #NUFC #Arsenal #HYPE,hype,"negative,anticipation"
3,#fuck #chelsea #love #arsenal http://t.co/EmNX...,love,"joy,positive"
4,Watching both #Arsenal &amp; #DerbyCounty #Foo...,ram,"negative,anticipation,anger"
5,Good start from the Arsenal #pressure #Arsenal,pressure,negative
6,Congrats to #mofarah on his 3rd major double #...,gold,positive
7,Come on #NUFC they are running at you! #Scum,scum,"disgust,negative"
8,So a perfectly good goal disallowed and this w...,conspiracy,fear
9,So a perfectly good goal disallowed against Li...,conspiracy,fear


In [9]:
# Save
useful_methods.DFtoCSV(dfNewHashEmolex, "/Users/Bya/Dropbox/Research/datas/TweetsPN/", "hash_emolex_all", index=False)